<a href="https://colab.research.google.com/github/michelup/TEST_SDA/blob/main/FINAL_PROJECT_Pokemons_DenseNET121.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import numpy as np
import json
import requests
import kagglehub
import os
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

import tensorflow as tf
import keras
from keras.src.applications.densenet import DenseNet121, preprocess_input
from keras.src.models import Sequential
from keras.src.utils import image_dataset_from_directory
from keras import layers, models
from keras.src.layers import Dense, Dropout

import matplotlib.pyplot as plt
import pandas as pd



from PIL import Image, ImageOps

from keras.applications import MobileNetV2
from keras.applications.mobilenet_v2 import preprocess_input
from keras.preprocessing import image_dataset_from_directory
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten
from keras.layers import Input, Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.models import Model, Sequential



In [14]:
#data preprocessing
  #handling missing values
  #data scaling
  #categorical features encoding
  #preparation or extraction of the test collection - test/train
  #data augmentation

#model architecture
  #input layer
  #hidden layer
  #output layer
  #neurons
  #activation functions
  #connection between neurons
  #parametres

#Loss function
#optimizer
#iteratice learning process
#Hyper parameter tunning
#Final testing










In [15]:
from google.colab import files
uploaded = files.upload()

Saving first-gen-pokemon.zip to first-gen-pokemon (1).zip


In [16]:
import zipfile

# nahraď názvem svého souboru
zip_path = 'first-gen-pokemon.zip'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('pokemon_data')

# ověř, co je uvnitř
print(os.listdir('pokemon_data'))
print(os.listdir('pokemon_data/dataset'))

['pokemons_gen1.csv', 'dataset']
['test', 'train']


In [17]:
train_dir = 'pokemon_data/dataset/train'
test_dir = 'pokemon_data/dataset/test'

In [18]:

#priprava dat

img_size = (224, 224)  #resampling na  224x224
batch_size = 32   #velikost batche 32


#definice trenovacich dat
train_dataset = image_dataset_from_directory(
    train_dir,
    label_mode='categorical',  #priradi label podle kategorie/slozek v adresari
    shuffle=True,
    image_size=img_size, # fotka bude upravena na velikost definovanou promennou v  img_size vyse
    batch_size=batch_size
)

#definice testovacich dat
test_dataset = image_dataset_from_directory(
    test_dir,
    label_mode='categorical',
    shuffle=False,
    image_size=img_size,
    batch_size=batch_size
)

Found 2249 files belonging to 151 classes.
Found 840 files belonging to 151 classes.


In [19]:
class_names = test_dataset.class_names #seznam tříd/názvů jednotlivých kategorií, odvozenych z názvů složek při načítání datasetu


In [20]:
train_dataset = train_dataset.map(lambda x, y: (preprocess_input(x), y))
test_dataset = test_dataset.map(lambda x, y: (preprocess_input(x), y))

In [21]:
# vytvoreni konvoluční neuronove síťe pro klasifikaci obrázků pomocí transfer learningu s předtrénovaným modelem DenseNet121
# model využívá silný předtrénovaný základ DenseNet121 pro extrakci vizuálních rysů a staví na něm vlastní klasifikátor pro nový úkol s 151 kategoriemi/pokemony
from tensorflow.keras import Sequential, layers
from tensorflow.keras.applications import DenseNet121


base_model = DenseNet121(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)

model = Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(151, activation='softmax')
])

In [22]:
model.compile(
    optimizer="adam",
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [23]:
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    'soubor.keras',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

#pripojit  google drive a ulozit checkpoint na muj  drive

In [ ]:
epoch_val = 20

history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=epoch_val,
    callbacks=[checkpoint_callback]
)

Epoch 1/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 23s/step - accuracy: 0.0261 - loss: 5.0273 
Epoch 1: val_accuracy improved from -inf to 0.02381, saving model to soubor.keras
71/71 ━━━━━━━━━━━━━━━━━━━━ 1894s 25s/step - accuracy: 0.0262 - loss: 5.0252 - val_accuracy: 0.0238 - val_loss: 31.1367
Epoch 2/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 22s/step - accuracy: 0.0956 - loss: 4.4297 
Epoch 2: val_accuracy improved from 0.02381 to 0.07143, saving model to soubor.keras
71/71 ━━━━━━━━━━━━━━━━━━━━ 1735s 24s/step - accuracy: 0.0958 - loss: 4.4277 - val_accuracy: 0.0714 - val_loss: 7.0355
Epoch 3/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 22s/step - accuracy: 0.1827 - loss: 3.7400 
Epoch 3: val_accuracy improved from 0.07143 to 0.16667, saving model to soubor.keras
71/71 ━━━━━━━━━━━━━━━━━━━━ 1736s 24s/step - accuracy: 0.1829 - loss: 3.7376 - val_accuracy: 0.1667 - val_loss: 4.1653
Epoch 4/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 22s/step - accuracy: 0.3144 - loss: 2.9366 
Epoch 4: val_accuracy improved from 0.16667 to 0.2785

In [ ]:
pd.DataFrame(history.history).plot()

In [ ]:
saved_model = keras.saving.load_model("soubor.keras")